In [2]:
import os
import numpy as np
from PIL import Image
from pprint import pprint


def load_images(folder):
    print(f"\n[INFO] Loading images from folder: {folder}")
    imgs = []

    for fname in os.listdir(folder):
        path = os.path.join(folder, fname)
        print(f"[LOAD] Reading image: {fname}")

        img = Image.open(path).convert("L")
        img_arr = np.array(img, dtype=np.float32)

        print(f"       Image shape: {img_arr.shape}")
        print(f"       Min pixel value: {img_arr.min()}, Max pixel value: {img_arr.max()}")

        imgs.append(img_arr)

    print(f"[INFO] Total images loaded from {folder}: {len(imgs)}")
    return imgs


def extract_blocks(img, block=8):
    H, W = img.shape
    print(f"\n[INFO] Extracting {block}x{block} blocks from image of size {H}x{W}")

    blocks = []
    block_count = 0

    for i in range(0, H, block):
        for j in range(0, W, block):
            b = img[i:i+block, j:j+block]

            if b.shape == (block, block):
                flat_block = b.reshape(-1)
                blocks.append(flat_block)

                if block_count < 3:
                    print(f"[BLOCK {block_count}] Location: ({i},{j})")
                    print(f"           Block shape: {b.shape}")
                    print(f"           Flattened length: {len(flat_block)}")
                    print(f"           First 8 values: {flat_block[:8]}")

                block_count += 1

    print(f"[INFO] Total blocks extracted: {block_count}")
    return np.array(blocks)


# ============================
# Load datasets
# ============================

train_images = load_images("TrainSet")
test_images  = load_images("TestSet")

print("\n============================")
print("[INFO] Processing TRAIN images")
print("============================")

train_blocks_list = []
for idx, img in enumerate(train_images):
    print(f"\n[TRAIN IMAGE {idx}]")
    blocks = extract_blocks(img, block=8)
    train_blocks_list.append(blocks)

train_blocks = np.concatenate(train_blocks_list) / 255.0

print("\n[INFO] TRAIN blocks summary")
print(f"Total train blocks: {train_blocks.shape[0]}")
print(f"Block vector length (n): {train_blocks.shape[1]}")
print(f"Value range after normalization: min={train_blocks.min()}, max={train_blocks.max()}")

print("\nSample TRAIN block (first one):")
pprint(train_blocks[0])


print("\n============================")
print("[INFO] Processing TEST images")
print("============================")

test_blocks_list = []
for idx, img in enumerate(test_images):
    print(f"\n[TEST IMAGE {idx}]")
    blocks = extract_blocks(img, block=8)
    test_blocks_list.append(blocks)

test_blocks = np.concatenate(test_blocks_list) / 255.0

print("\n[INFO] TEST blocks summary")
print(f"Total test blocks: {test_blocks.shape[0]}")
print(f"Block vector length (n): {test_blocks.shape[1]}")
print(f"Value range after normalization: min={test_blocks.min()}, max={test_blocks.max()}")

print("\nSample TEST block (first one):")
pprint(test_blocks[0])



[INFO] Loading images from folder: TrainSet
[LOAD] Reading image: train (0).jpg
       Image shape: (256, 256)
       Min pixel value: 0.0, Max pixel value: 255.0
[LOAD] Reading image: train (1).jpg
       Image shape: (256, 256)
       Min pixel value: 6.0, Max pixel value: 255.0
[LOAD] Reading image: train (10).jpg
       Image shape: (256, 256)
       Min pixel value: 35.0, Max pixel value: 217.0
[LOAD] Reading image: train (11).jpg
       Image shape: (256, 256)
       Min pixel value: 34.0, Max pixel value: 210.0
[LOAD] Reading image: train (12).jpg
       Image shape: (256, 256)
       Min pixel value: 16.0, Max pixel value: 255.0
[LOAD] Reading image: train (13).jpg
       Image shape: (256, 256)
       Min pixel value: 0.0, Max pixel value: 225.0
[LOAD] Reading image: train (14).jpg
       Image shape: (256, 256)
       Min pixel value: 0.0, Max pixel value: 255.0
[LOAD] Reading image: train (15).jpg
       Image shape: (256, 256)
       Min pixel value: 0.0, Max pixel value: 

In [5]:
def relu(x): # activation function for Output layer
    return np.maximum(0, x)

def relu_deriv(x): # for backpropagation to update weights
    return (x > 0).astype(float)

def sigmoid(x): # activation function for Hidden layer
    return 1 / (1 + np.exp(-x))

def sigmoid_deriv(x):
    s = sigmoid(x)
    return s * (1 - s)


In [6]:
def prepare_data_by_block(images_list, block_size, verbose=False):
    if verbose:
        print(f"[INFO] Preparing data with block size: {block_size}x{block_size}")
        print(f"[INFO] Number of images: {len(images_list)}")

    all_blocks = []

    for idx, img in enumerate(images_list):
        if verbose:
            print(f"\n[IMAGE {idx}] shape: {img.shape}")

        blocks = extract_blocks(img, block=block_size)

        if verbose:
            print(f"  Extracted blocks: {blocks.shape}")
            print(f"  Sample block (first): {blocks[0][:8]}")

        all_blocks.append(blocks)

    data = np.concatenate(all_blocks) / 255.0

    if verbose:
        print("\n[INFO] Final dataset summary")
        print(f"  Total blocks: {data.shape[0]}")
        print(f"  Block vector length: {data.shape[1]}")
        print(f"  Value range: min={data.min()}, max={data.max()}")

    return data


In [7]:
def MSE(X, Y):
    return np.mean((X - Y)**2)

def PSNR(mse):
    return 10 * np.log10(1 / mse)

In [9]:
class MLP_Autoencoder_Improved:
    
    # MODIFIED: Added use_momentum flag with default=True
    def __init__(self, input_dim=64, hidden_dim=16, use_momentum=True, beta=0.9):
        
        # ... (W1, b1, W2, b2 initialization remains the same)
        self.W1 = np.random.randn(input_dim, hidden_dim) * np.sqrt(2/input_dim)
        self.b1 = np.zeros((1, hidden_dim))
        self.W2 = np.random.randn(hidden_dim, input_dim) * np.sqrt(2/hidden_dim)
        self.b2 = np.zeros((1, input_dim))
        # ... (vW, vb initialization remains the same)

        # MODIFIED: Momentum States and Flag
        self.vW1 = np.zeros_like(self.W1)
        self.vb1 = np.zeros_like(self.b1)
        self.vW2 = np.zeros_like(self.W2)
        self.vb2 = np.zeros_like(self.b2)
        
        # If use_momentum is False, set beta to 0, which effectively turns off momentum.
        self.beta = beta if use_momentum else 0.0 
        self.use_momentum = use_momentum # For tracking in the report
        
    def forward(self, X, verbose=False):
        if verbose:
            print("[FORWARD] Input shape:", X.shape)

        self.X = X

        self.z1 = X @ self.W1 + self.b1
        self.a1 = relu(self.z1)

        if verbose:
            print("[ENCODER] z1 mean:", self.z1.mean())
            print("[ENCODER] a1 mean:", self.a1.mean())

        self.z2 = self.a1 @ self.W2 + self.b2
        self.a2 = sigmoid(self.z2)

        if verbose:
            print("[DECODER] z2 mean:", self.z2.mean())
            print("[OUTPUT] a2 range:", self.a2.min(), self.a2.max())

        return self.a2

    
    # MODIFIED: Cleaned up momentum logic in backward
    def backward(self, lr=0.05, verbose=False):
        m = self.X.shape[0]
    
        dz2 = (self.a2 - self.X) * sigmoid_deriv(self.z2)
        dW2 = self.a1.T @ dz2 / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m
    
        dz1 = (dz2 @ self.W2.T) * relu_deriv(self.z1)
        dW1 = self.X.T @ dz1 / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m
    
        if verbose:
            print("[BACKWARD] dW1 norm:", np.linalg.norm(dW1))
            print("[BACKWARD] dW2 norm:", np.linalg.norm(dW2))
            print("[MOMENTUM]", "ON" if self.use_momentum else "OFF")
    
        self.vW2 = self.beta * self.vW2 + dW2
        self.vb2 = self.beta * self.vb2 + db2
        self.vW1 = self.beta * self.vW1 + dW1
        self.vb1 = self.beta * self.vb1 + db1
    
        self.W2 -= lr * self.vW2
        self.b2 -= lr * self.vb2
        self.W1 -= lr * self.vW1
        self.b1 -= lr * self.vb1


In [19]:
def train_autoencoder_minibatch(model, X_train, X_test, epochs=5, lr=0.005, batch_size=64):
    avg_loss_list = []
    test_psnr_list = []
    data_size = X_train.shape[0]
    indices = np.arange(data_size)
    
    for ep in range(epochs):
        np.random.shuffle(indices)
        X_shuffled = X_train[indices]
        epoch_loss = 0
        num_batches = 0
        
        for i in range(0, data_size, batch_size):
            X_batch = X_shuffled[i:i + batch_size]
            print(f"\n[Batch {num_batches+1}] Input shape: {X_batch.shape}")
            
            out_batch = model.forward(X_batch, verbose=True)
            loss_batch = MSE(out_batch, X_batch)
            print(f"Batch MSE: {loss_batch:.6f}")
            
            model.backward(lr, verbose=True)
            
            epoch_loss += loss_batch
            num_batches += 1
        
        avg_loss = epoch_loss / num_batches
        out_test = model.forward(X_test, verbose=False)
        mse_test = MSE(out_test, X_test)
        psnr_test = PSNR(mse_test)
        
        avg_loss_list.append(avg_loss)
        test_psnr_list.append(psnr_test)
        
        print(f"\n[Epoch {ep+1}] Avg Train MSE={avg_loss:.6f} | Test PSNR={psnr_test:.2f} dB")
    
    return avg_loss_list, test_psnr_list


In [18]:
def reconstruct_image(model, img, block_size=8, max_blocks=10):
    """
    Reconstructs an image block-by-block using the autoencoder
    and prints detailed information for the first few blocks.
    
    Args:
        model: Trained autoencoder model
        img: Input grayscale image (H x W)
        block_size: Size of each square block
        max_blocks: Number of blocks to print info for (to avoid flooding)
        
    Returns:
        Reconstructed image as uint8
    """
    
    H, W = img.shape
    blocks = extract_blocks(img, block=block_size) / 255.0
    out = model.forward(blocks) * 255.0
    out = out.astype(np.uint8)
    
    new_img = np.zeros((H, W), dtype=np.uint8)
    bi = 0
    
    for i in range(0, H, block_size):
        for j in range(0, W, block_size):
            block_original = img[i:i+block_size, j:j+block_size]
            block_recon = out[bi].reshape(block_size, block_size)
            
            new_img[i:i+block_size, j:j+block_size] = block_recon
            
            # Verbose output for first few blocks
            if bi < max_blocks:
                mean_orig = block_original.mean()
                mean_recon = block_recon.mean()
                diff = np.mean(np.abs(block_original - block_recon))
                
                print(f"[Block {bi}] Location: ({i},{j})")
                print(f"    Original mean: {mean_orig:.2f}")
                print(f"    Reconstructed mean: {mean_recon:.2f}")
                print(f"    Avg absolute difference: {diff:.2f}")
            
            bi += 1
    
    print(f"\nReconstructed full image with {bi} blocks.")
    return new_img


In [12]:
import matplotlib.pyplot as plt
from PIL import Image

# ... (سایر توابع) ...

def show_reconstruction(model, img, block_size):
    """
    Shows image reconstruction using the trained model,
    using the correct block_size corresponding to the model.
    """
    # 1. Extract blocks using the correct size
    blocks = extract_blocks(img, block=block_size) / 255.0 
    
    # 2. Forward pass and denormalize
    out = model.forward(blocks) * 255.0
    out = out.astype(np.uint8)
    
    # 3. Reconstruct image from blocks
    H, W = img.shape
    rec = np.zeros((H, W), dtype=np.uint8)

    k = 0
    # The reconstruction logic should also use block_size
    for i in range(0, H, block_size):
        for j in range(0, W, block_size):
            # Check if block exists (to handle edges if image size is not divisible)
            if k < len(out):
                 rec[i:i+block_size, j:j+block_size] = out[k].reshape(block_size, block_size)
                 k += 1

    # 4. Display results
    plt.figure(figsize=(8,4))
    
    plt.subplot(1,2,1)
    plt.title("Original")
    plt.imshow(img, cmap='gray')
    plt.axis(False)

    plt.subplot(1,2,2)
    plt.title(f"Reconstructed (B={block_size} H={model.W1.shape[1]})")
    plt.imshow(rec, cmap='gray')
    plt.axis(False)

    plt.show() # این دستور مسئول نمایش خروجی بصری است.
    return rec

In [13]:
def plot_curves(results):
    plt.figure(figsize=(12,5))

    # Train MSE
    plt.subplot(1,2,1)
    for k in results:
        plt.plot(results[k][0], label=f"hidden={k}")
    plt.title("Train MSE")
    plt.xlabel("Epoch")
    plt.ylabel("MSE")
    plt.legend()

    # Test PSNR
    plt.subplot(1,2,2)
    for k in results:
        plt.plot(results[k][1], label=f"hidden={k}")
    plt.title("Test PSNR")
    plt.xlabel("Epoch")
    plt.ylabel("PSNR (dB)")
    plt.legend()

    plt.show()


In [14]:
def train_with_visual(model, X_train, img_sample, epochs=20, lr=0.01):
    frames = []

    for ep in range(epochs):
        out = model.forward(X_train)
        model.backward(X_train, lr)

        # reconstruct one sample image each epoch
        rec = reconstruct_image(model, img_sample)
        frames.append(rec)

        print(f"Epoch {ep+1}/{epochs}")

    return frames


In [15]:
def show_frames(frames):
    for i, frame in enumerate(frames):
        plt.imshow(frame, cmap='gray')
        plt.title(f"Epoch {i+1}")
        plt.axis(False)
        plt.show()


In [16]:
def summarize_results(results_dict):
    """Generates a summary table of final PSNR values for the report."""
    
    # Extracting Block Sizes (Rows) and Hidden Dims (Columns)
    blocks = sorted(list(set([k[0] for k in results_dict.keys()])))
    hiddens = sorted(list(set([k[1] for k in results_dict.keys()])))

    table = "## Average PSNR Summary (dB)\n"
    table += "| Block Size | " + " | ".join([f"Hidden={h}" for h in hiddens]) + " |\n"
    table += "| :---: | " + " | ".join([":---:"] * len(hiddens)) + " |\n"

    for b in blocks:
        row = f"| {b}x{b} |"
        for h in hiddens:
            key = (b, h)
            # CHECK ADDED: Only access the key if it exists in the dictionary
            if key in results_dict:
                psnr_value = results_dict[key]['final_psnr']
                row += f" {psnr_value:.2f} |"
            else:
                row += f" N/A |" # Show N/A for skipped cases
        table += row + "\n"
        
    return table

In [17]:
# --- 1. Preparation (Assuming all helper functions and classes are defined) ---
# --- Block Sizes: 4, 8, 16. Hidden Dims: 4, 16, 32. ---
# --- Training Parameters ---
HIDDEN_DIMS = [4, 16, 32]
BLOCK_SIZES = [4, 8, 16]
LEARNING_RATE = 0.005 # Adjusted to a safer value for stability
EPOCHS = 100
BATCH_SIZE = 128
INPUT_IMG_SAMPLE = test_images[0] # Image for visualization

combined_results = {} # Stores all 3x3 combinations
momentum_results = {} # Stores Momentum ON/OFF comparison

# --- 2. Phase 1: 3x3 Experiments (Hidden Dim vs Block Size) ---
# (The code block from the previous turn)
for block_s in BLOCK_SIZES:
    input_dim = block_s * block_s
    train_data = prepare_data_by_block(train_images, block_s)
    test_data = prepare_data_by_block(test_images, block_s)
    
    for hid in HIDDEN_DIMS:
        if hid > input_dim:
            print(f"Skipping B={block_s}x{block_s}, H={hid}")
            continue
            
        print(f"\n[EXP] Training B={block_s}x{block_s}, H={hid} (Momentum ON)")
        model = MLP_Autoencoder_Improved(input_dim=input_dim, hidden_dim=hid, use_momentum=True)
        
        train_c, test_c = train_autoencoder_minibatch(model, train_data, test_data, epochs=EPOCHS, lr=LEARNING_RATE, batch_size=BATCH_SIZE)
        
        combined_results[(block_s, hid)] = {
            'final_psnr': test_c[-1],
            'model': model,
            'train_curve': train_c,
            'test_curve': test_c
        }
        print(f"Final PSNR: {test_c[-1]:.2f} dB")


# --- 3. Phase 2: Momentum Comparison (B=8, H=16) ---
B_MOMENTUM, H_MOMENTUM = 8, 16
print(f"\n--- Phase 2: Momentum Comparison (B=8x8, H=16) ---")

train_data_mom = prepare_data_by_block(train_images, B_MOMENTUM)
test_data_mom = prepare_data_by_block(test_images, B_MOMENTUM)

# Momentum ON
model_on = MLP_Autoencoder_Improved(input_dim=64, hidden_dim=16, use_momentum=True, beta=0.9)
train_c_on, test_c_on = train_autoencoder_minibatch(model_on, train_data_mom, test_data_mom, epochs=EPOCHS, lr=LEARNING_RATE, batch_size=BATCH_SIZE)
momentum_results['Momentum ON'] = (train_c_on, test_c_on)
print(f"Momentum ON Final PSNR: {test_c_on[-1]:.2f} dB")

# Momentum OFF
model_off = MLP_Autoencoder_Improved(input_dim=64, hidden_dim=16, use_momentum=False)
train_c_off, test_c_off = train_autoencoder_minibatch(model_off, train_data_mom, test_data_mom, epochs=EPOCHS, lr=LEARNING_RATE, batch_size=BATCH_SIZE)
momentum_results['Momentum OFF'] = (train_c_off, test_c_off)
print(f"Momentum OFF Final PSNR: {test_c_off[-1]:.2f} dB")


# --- 4. Report Generation (Visualization) ---

# A. Summary Table (Comparison of all PSNR)
print("\n" + summarize_results({k: v for k, v in combined_results.items()}))

# B. Learning Curves (Rend-e Yadgiri) for all 3x3 cases
print("\n--- Plotting All 3x3 Learning Curves (Train MSE and Test PSNR) ---")
# Reformat the results for the existing plot_curves function
plot_data_3x3 = {f"{k[0]}x{k[0]} H={k[1]}": (v['train_curve'], v['test_curve']) for k, v in combined_results.items()}
plot_curves(plot_data_3x3) # Requires adaptation in plot_curves to handle dictionary of tuples

# C. Momentum Comparison Curve
print("\n--- Plotting Momentum Comparison Curve (Test PSNR) ---")
# This plot compares only the Test PSNR for ON vs OFF
plot_curves(momentum_results)


# D. Image Reconstructions (Tasaweer Bazsazi shode)
INPUT_IMG_SAMPLE = test_images[0]

# 1. Best performing model (B=8, H=16)
best_model_key = (8, 16) 
if best_model_key in combined_results:
    print(f"\nReconstruction for BEST Model (B=8x8, H=16):")
    # MODIFIED CALL: Passing block_size=8
    show_reconstruction(combined_results[best_model_key]['model'], INPUT_IMG_SAMPLE, block_size=8)

# 2. Worst performing model (B=16, H=4)
worst_model_key = (16, 4) 
if worst_model_key in combined_results:
    print(f"\nReconstruction for WORST Model (B=16x16, H=4):")
    # MODIFIED CALL: Passing block_size=16
    show_reconstruction(combined_results[worst_model_key]['model'], INPUT_IMG_SAMPLE, block_size=16)

# 3. Momentum ON vs OFF (Both use B=8)
print("\nReconstruction for Momentum ON (B=8x8):")
show_reconstruction(model_on, INPUT_IMG_SAMPLE, block_size=8)
print("\nReconstruction for Momentum OFF (B=8x8):")
show_reconstruction(model_off, INPUT_IMG_SAMPLE, block_size=8)


[INFO] Extracting 4x4 blocks from image of size 256x256
[BLOCK 0] Location: (0,0)
           Block shape: (4, 4)
           Flattened length: 16
           First 8 values: [249. 249. 250. 251. 249. 249. 250. 251.]
[BLOCK 1] Location: (0,4)
           Block shape: (4, 4)
           Flattened length: 16
           First 8 values: [252. 254. 254. 255. 252. 253. 254. 255.]
[BLOCK 2] Location: (0,8)
           Block shape: (4, 4)
           Flattened length: 16
           First 8 values: [253. 253. 253. 253. 253. 253. 253. 253.]
[INFO] Total blocks extracted: 4096

[INFO] Extracting 4x4 blocks from image of size 256x256
[BLOCK 0] Location: (0,0)
           Block shape: (4, 4)
           Flattened length: 16
           First 8 values: [111. 110. 119. 120. 113. 109. 114. 123.]
[BLOCK 1] Location: (0,4)
           Block shape: (4, 4)
           Flattened length: 16
           First 8 values: [121. 119. 106. 101. 134. 135. 119. 107.]
[BLOCK 2] Location: (0,8)
           Block shape: (4, 4)
   

NameError: name 'train_autoencoder_minibatch' is not defined